# TEST-SCRIPT for string distance measures and alignments

This package implements sequence -2- sequence aligning and matching for character strings.
It is an implementation of general DP with SUB/INS/DEL moves and with local distance being the equality test (x_i == y_j). 
The matching can be done at the character or word level by invoking different tokenization options.
   
Applications include aligning strings, measuring error rates in ASR, finding typos, ...

All algorithms have a computational cost O(N^2) as they are variants on the Dynamic Programming principle.
Memory cost is O(N^3) for those routines allowing for backtracking and aligning.  Memore costs is O(N^2) for the routines
that strictly compute a distance.

The term 'measure' is intentional as these measures are not 'metrics' in a mathematical sense.

- levenshtein():
   + computes the Levenshtein distance, i.e. #S+#I+#D
   + this is a single (forward) pass algorithm that only yields the composite distance, no alignment, no backtracking 
      
- edit_distance(): 
    - computes the weighted edit distance
    - allowing Substitutions, Insertion, Deletions and Compounds. 
    - the detailed edit counts
    - optionally an alignment is computed via backtracking (forward + backward pass) 
    
- align():
    - returns the sequence alignment

- levenshtein() and edit_distance() 
    - accept a single pair of strings or a set of matching pairs 
    - they return
        - edit distance (composite over the full set)
        - a results structure with details
        
- levenshtein_t() and edit_distance_t()
    - are underlying routines that take a single pair of tokens as input
       

## levenshtein() and edit_distance() details 
- TOKENIZATION: text strings are first converted to a list of tokens before using these modules.   Tokenization options are:
    - TOKEN = None (default): string is converted to list of characters
    - TOKEN = "WORD"
    - TOKEN = "CHAR"
- Compounding: The edit_distance() routine allows for compounding if **CMPND** characters are defined.  By default there is no compounding and CMPND=[].  Compounding is naive by allowing 2 word compounds only and by defining compounding rules by compounding characters, including '' for cross white space compounding.  The most common option is to allow for simple and dash compounding by defining CMPND=\['','-'\]

- nomenclature
  + number_of_edits: the number of edits (sum of substitutions, insertions and deletions) making abstraction of compounds
  + weighted_edit_distance: weighted sum of edits (possibly including compounds)
  + hypothesis, reference:   edits are applied on the reference to yield the hypothesis
  + alignment: an alignment made between reference and hypothesis'
  + trellis: 2D arrangement of alignment/alignment scores used in the DP
  
- flags and options:
  + VERBOSE:  to print intermediate results
  + ALIGN:       to do backtracking and return an alignment on top of the scores
  + TOKEN:    tokenization option
    
- results:   
Both routines return a dictionary of results. The elements depend on the selected routine and options and may include:
>    + 'total': number of edits
>    + 'ins': number of insertions
>    + 'sub': number of substitutions
>    + 'del': number of deletions
>    + 'comp': number of compounds
>    + 'err': error rate (in %)
>    + 'hyp_tokens': tokens in hypothesis
>    + 'ref_tokens': tokens in reference
>    + 'utterances': number of processed utterances
>    + 'errors': list of errors
>    + 'align' : list of dataframes with utterance alignments
>    + 'edits' : list of edits ('H','S','I','D','C')
 

In [1]:
# do all the imports
import numpy as np
import pandas as pd
import timeit
import sys, os

sys.path.insert(0,os.path.abspath('..'))
import evalign as eva

## Alignment of strings with **align()**
Aligning sequence is one of the primary goals of the available sequence-2-sequence matching routines.   
**align()** is a convenience routine that simply returns such alignment. Under the hood it is a wrapper around edit_distance() .   
It is possible to pass extra \*\*kwargs arguments to the underlying routine 

In [2]:
utt1 = "recognize speech"
utt2 = "wreck a nice beach"
eva.align(utt1,utt2,EPS="*")

[('*', 'w'),
 ('r', 'r'),
 ('e', 'e'),
 ('c', 'c'),
 ('*', 'k'),
 ('*', ' '),
 ('o', 'a'),
 ('g', ' '),
 ('n', 'n'),
 ('i', 'i'),
 ('z', 'c'),
 ('e', 'e'),
 (' ', ' '),
 ('s', '*'),
 ('p', 'b'),
 ('e', 'e'),
 ('e', 'a'),
 ('c', 'c'),
 ('h', 'h')]

In [3]:
utt1 = "recognize a speech"
utt2 = "wreck a nice beach"
eva.align(utt1,utt2,TOKEN="WORD",EPS="*")

[('recognize', 'wreck'), ('a', 'a'), ('*', 'nice'), ('speech', 'beach')]

## Levenshtein Distance

The Levenshtein() distance computes the number of elementary edits (SUB, INS, DEL) to convert one sequence into another.  It is the simplest of all sequence distance measures.  Moreover it is a symmetric distance measure, hence x and y can be reversed.   
The levenshtein() function takes strings or list of strings as inputs.

In [4]:
help(eva.levenshtein)

Help on function levenshtein in module evalign.distance:

levenshtein(x, y, TOKEN=None, **kwargs)
     Finds the symmetric Levenshtein distance (sum of SUB/INS/DEL) between two strings of across a corpus 
     of x-y pairs.    x and y position are irrelevant.
     There is no backtracking, and no separate tracking of SUB/INS/DEL 
     
     Note:
     + for token counts 'x' is designed as 'hyp and 'y' as 'ref'
     + for error rate computation: average length of x and y is used in the division
     
     Parameters
     ----------
         x : str or list of str
    
         y : str or list of str
    
         TOKEN: None or str ("WORD" or "CHAR")
             see utils.tokenizer()
         
         **kwargs:
             passed to levenshtein()
     
     Returns
    --------
         lev_dist:     int
             Levenshtein Distance (combined)
             
         results:      a dictionary with        
                         'hyp_tokens' : (int) number of tokens in hypothes

In [5]:
x = "bleker"
y = "broek" 
lev_dist,_ = eva.levenshtein(x,y)
print("Levensthein Distance (\"%s\",\"%s\"): %d " %(x,y,lev_dist) )
assert(lev_dist==4)

Levensthein Distance ("bleker","broek"): 4 


In [6]:
# O(N^2) time behavior of Levenshtein and other DP routines
print("Levenshtein and other DP routines have roughly a O(N^2) behavior")
print("Execution time for moderate sized strings ----- ")
%time ld1= eva.levenshtein(x*10,y*10)
print("Execution time for strings 10 times the original size")
%time ld10 = eva.levenshtein(x*100,y*100)
print("Observe execution time is roughly *100, i.e. O(N^2) behavior")
#ld1, ld10

Levenshtein and other DP routines have roughly a O(N^2) behavior
Execution time for moderate sized strings ----- 
Wall time: 7.11 ms
Execution time for strings 10 times the original size
Wall time: 546 ms
Observe execution time is roughly *100, i.e. O(N^2) behavior


## 2. CHARACTER and WORD TOKENS

Sequence-2-sequence matching is used both for measuring a distance between 2 sequences and for aligning them.   
A Levenshtein distance measures with how many edits when sequence is converted into the other.
The distance itself may be computed with a forward pass only (as is the case in this implementation of Levenshtein). 
An alignment is obtained after backtracking.  

The sequence matching routines levenshtein(), align() and edit_distance() all take both **list of strings** and **strings** as inputs that pass tokenized versions of the inputs to underlying levenshtein_t() and edit_distance_t() routines.

Tokenization is done by the utils.tokenizer() routine, where tokenization is done based on the TOKEN argument.
- TOKEN is None : string -> list conversion
- TOKEN=="WORD": word tokenization (white space disappears)
- TOKEN=="CHAR": character tokenization (after removal of extraneous white space)

In [7]:
#help(eva.utils.tokenizer)

In [8]:
x = "Yesterday I read a nice book"
y =  "Jeremy got a nice book again"
#
lev_dist, _ = eva.levenshtein(x,y)
align_char = eva.align(x,y)
print("Character Levensthein Distance (\"%s\",\"%s\"): %d " %(x,y,lev_dist) )
print(align_char)
#
lev_dist, _ = eva.levenshtein(x,y,TOKEN="WORD")
align_word = eva.align(x,y,TOKEN="WORD")
print("Word Levensthein Distance (\"%s\",\"%s\"): %d " %(x,y,lev_dist) )
print(align_word)

Character Levensthein Distance ("Yesterday I read a nice book","Jeremy got a nice book again"): 18 
[('Y', '_'), ('e', '_'), ('s', '_'), ('t', 'J'), ('e', 'e'), ('r', 'r'), ('d', 'e'), ('a', 'm'), ('y', 'y'), (' ', '_'), ('I', '_'), (' ', ' '), ('r', '_'), ('e', 'g'), ('a', 'o'), ('d', 't'), (' ', ' '), ('a', 'a'), (' ', ' '), ('n', 'n'), ('i', 'i'), ('c', 'c'), ('e', 'e'), (' ', ' '), ('b', 'b'), ('o', 'o'), ('o', 'o'), ('k', 'k'), ('_', ' '), ('_', 'a'), ('_', 'g'), ('_', 'a'), ('_', 'i'), ('_', 'n')]
Word Levensthein Distance ("Yesterday I read a nice book","Jeremy got a nice book again"): 4 
[('Yesterday', '_'), ('I', 'Jeremy'), ('read', 'got'), ('a', 'a'), ('nice', 'nice'), ('book', 'book'), ('_', 'again')]


In [9]:
# TOKENIZATION: string -> list
y = "   bleker"
x = "broek. " 
lev_dist, _ = eva.levenshtein(x,y)
print("RAW Character Levensthein Distance (\"%s\",\"%s\"): %d " %(x,y,lev_dist) )
align_char = eva.align(x,y,EPS='*')
print("with alignment: ")
print(align_char)
# TOKENIZATION: string -> meaningfull characters
lev_dist, _ = eva.levenshtein(x,y,TOKEN="CHAR")
print("Character Levensthein Distance (\"%s\",\"%s\"): %d " %(x,y,lev_dist) )
align_char = eva.align(x,y,TOKEN="CHAR",EPS='*')
print("with alignment: ")
print(align_char)

RAW Character Levensthein Distance ("broek. ","   bleker"): 7 
with alignment: 
[('*', ' '), ('*', ' '), ('*', ' '), ('b', 'b'), ('r', '*'), ('o', 'l'), ('e', 'e'), ('k', 'k'), ('.', 'e'), (' ', 'r')]
Character Levensthein Distance ("broek. ","   bleker"): 7 
with alignment: 
[('*', ' '), ('*', ' '), ('*', ' '), ('b', 'b'), ('r', '*'), ('o', 'l'), ('e', 'e'), ('k', 'k'), ('*', 'e'), (' ', 'r')]


## 2. Edit Distance
**edit_distance()** is a more versatile sequence distance metric than Levenshtein.  
It minimizes the weighted sum of edits: **wS x #SUB + wI x #INS + wD x #DEL**.   
Within the *evalign* package edit_distance() is the sequence matching routine of choice to perform  alignment, error rate computations, ...

In practice more often than not the total number of edits in edit_distance is identical to Levenshtein.
However, levenshtein() often leads to ambiguous alignments and specific edit counts which are dependent on arbitrary code variations.
By using slightly non-uniform weights in edit_distance() almost all ambiguities are resolved in a natural way.

**edit_distance(x,y)** computes the distance between **x as hypothesis** and **y as reference**.  
- One needs to distinguish 'reference' and 'hypothesis' explicitly as the weighted edit_distance is not symmetric by definition (though in practice it often is) 
- EDITS are defined with respect to the REFERENCE
- 

This module returns a results dictionary,including:
- *total*:  the number of edits required to match both sequences subject to the weighted edit criterion
- *sub*,*ins*,*del*: the #SUB, #INS, #DEL
- *edit_dist*: the weighted edit distance measure
- ...

In [10]:
tst = "work" # "recognizer"    # "to recognize speech"
ref = "sword" #"wreck a nice"  # "to wreck a nice beach"
#
print("String Matching using Weighted Edit Distance")
x = eva.tokenizer(tst,TOKEN="CHAR")
y = eva.tokenizer(ref,TOKEN="CHAR")
print(x,'vs.',y)
eva.edit_distance(x,y)

String Matching using Weighted Edit Distance
['w', 'o', 'r', 'k'] vs. ['s', 'w', 'o', 'r', 'd']


(4.4,
 {'total': 4,
  'sub': 4,
  'ins': 0,
  'del': 0,
  'edit_dist': 4.4,
  'err': 100.0,
  'hyp_tokens': 4,
  'ref_tokens': 4,
  'utterances': 4})

In [11]:
# edits and edit-distance can change when modifying the weights
x = "work" # "recognizer"    # "to recognize speech"
y = "sword" #"wreck a nice"  #  "to wreck a nice beach"
#
print("String Matching using Weighted Edit Distance")
print(x,'vs.',y)

wI = 1.
wD = 1.
for wS in [1., 2.5 ]:
    _,results = eva.edit_distance(x,y,wI=wI,wD=wD,wS=wS)
    #
    print("\nWeights :",[wS, wI, wD])
    print("Weighted Edit Distance: %.2f " % results['edit_dist'])
    eva.pp_results(results)

String Matching using Weighted Edit Distance
work vs. sword

Weights : [1.0, 1.0, 1.0]
Weighted Edit Distance: 2.00 
Error Rate: 40.00% 
Error Details: #S=1 #I=0 #D=1
Edit Distance: 2.00 
Tokens (HYP): 4    (REF): 5 
Utterances: 1

Weights : [2.5, 1.0, 1.0]
Weighted Edit Distance: 3.00 
Error Rate: 60.00% 
Error Details: #S=0 #I=1 #D=2
Edit Distance: 3.00 
Tokens (HYP): 4    (REF): 5 
Utterances: 1


### Alignments and Edits from **edit_distance()**
With the **ALIGN** flag turned on an **alignment of (x,y)** as well as a **list of edits** are added to the results dictionary

In [12]:
x = "to recognize speech"
y =  "to wreck a nice beach"
print("Character Level Matchng and Alignment")
#
_,results = eva.edit_distance(x,y,ALIGN=True)
eva.pp_results(results)
print(results['align'])
print(results['edits'])
#
print("\nWord Level Matching and Alignment")
_,results = eva.edit_distance(x,y,TOKEN="WORD",ALIGN=True)
eva.pp_results(results)
print(results['align'])
print(results['edits'])

Character Level Matchng and Alignment
Error Rate: 42.86% 
Error Details: #S=5 #I=1 #D=3
Edit Distance: 9.50 
Tokens (HYP): 19    (REF): 21 
Utterances: 1
[('t', 't'), ('o', 'o'), (' ', ' '), ('_', 'w'), ('r', 'r'), ('e', 'e'), ('c', 'c'), ('_', 'k'), ('_', ' '), ('o', 'a'), ('g', ' '), ('n', 'n'), ('i', 'i'), ('z', 'c'), ('e', 'e'), (' ', ' '), ('s', '_'), ('p', 'b'), ('e', 'e'), ('e', 'a'), ('c', 'c'), ('h', 'h')]
['H', 'H', 'H', 'D', 'H', 'H', 'H', 'D', 'D', 'S', 'S', 'H', 'H', 'S', 'H', 'H', 'I', 'S', 'H', 'S', 'H', 'H']

Word Level Matching and Alignment
Error Rate: 80.00% 
Error Details: #S=2 #I=0 #D=2
Edit Distance: 4.20 
Tokens (HYP): 3    (REF): 5 
Utterances: 1
[('to', 'to'), ('_', 'wreck'), ('_', 'a'), ('recognize', 'nice'), ('speech', 'beach')]
['H', 'D', 'D', 'S', 'S']


### **edit_distance()** with Compounding
The edit_distance() module has an option for accepting naive compounding.  In practice a small edit weight (default=0.2) is assigned to a compound match and compound matches are not counted in the total number of edits nor error rate

Naive compounding in this context is defined as simple 2-word compounding with all acceptable compounding characters are specified in a CMPND list ; e.g. CMPND = \['','-'\] (glueing allowed over white space or dash).  Compounding is allowed equally in test and reference.

It is noted that if more sophisticated compounding is desired, that this should be tackled by preprocessing instead of the matching routine.

An extra complicating factor for computing error rates, when allowing for compounding is that the sequence lengths of test and reference may change due to compounding.  As in practice this is more frequent on the test (eg ASR output) and (very) rare on the reference we consider this a minor issue.   

In [13]:
y = "blackboards and filterbanks are ..."
x =  "black boards and filter banks"
#
print("Weighted Edit Distance allowing for Compounds")
_,results = eva.edit_distance(x,y,TOKEN="WORD",ALIGN=True,CMPND=[''])
#
eva.pp_results(results)
print(results['align'])
assert(results['total']==2)

Weighted Edit Distance allowing for Compounds
Error Rate: 40.00% 
Error Details: #S=0 #I=0 #D=2
Accepted Compounds: #C=2
Edit Distance: 2.40 
Tokens (HYP): 5    (REF): 5 
Utterances: 1
[('black+boards', 'blackboards'), ('and', 'and'), ('filter+banks', 'filterbanks'), ('_', 'are'), ('_', '...')]


In [14]:
y = "ik hou er van naar luister- en kamer-muziek te luisteren"
x =  "ik hou ervan naar luister en kamer muziek te blijven luisteren"
#
print("Weighted Edit Distance with Regular and Dash Compounding")
_,results = \
    eva.edit_distance(x,y,TOKEN="WORD",ALIGN=True,CMPND=['','-'])
#
eva.pp_results(results)
print(results['align'])
print(results['edits'])
#eva.print_align(results['align'][0])
assert(results['total']==2)

Weighted Edit Distance with Regular and Dash Compounding
Error Rate: 20.00% 
Error Details: #S=1 #I=1 #D=0
Accepted Compounds: #C=2
Edit Distance: 2.50 
Tokens (HYP): 11    (REF): 10 
Utterances: 1
[('ik', 'ik'), ('hou', 'hou'), ('ervan', 'er+van'), ('naar', 'naar'), ('luister', 'luister-'), ('en', 'en'), ('kamer+muziek', 'kamer-muziek'), ('te', 'te'), ('blijven', '_'), ('luisteren', 'luisteren')]
['H', 'H', 'C', 'H', 'S', 'H', 'C', 'H', 'I', 'H']


### Corpus Processing
The core routines process single strings or single lists of tokens.
Conglomerate versions , working with multiple utterances or lists, are available in the modules with extension "\_x"
These expanded versions always yield a result structure

In [15]:
y = ["   beekje","zandbak",""]
x = ["broek " , "brokken maken",""]
print("Levensthein Distance on a per sentence basis")
for xx,yy in zip(x,y):
    lev_dist,_ = eva.levenshtein(xx,yy)
    print("(%s vs %s):  %d " %(xx,yy,lev_dist))    

_,lev_results = eva.levenshtein(x,y)
print("Levenshtein Distance (corpus) :  %d " %lev_results['total'])
assert(lev_results['total']==17)

Levensthein Distance on a per sentence basis
(broek  vs    beekje):  7 
(brokken maken vs zandbak):  10 
( vs ):  0 
Levenshtein Distance (corpus) :  17 


C:\Users\compi\Nextcloud\github\evalign\evalign\distance.py:66: RuntimeWarning: invalid value encountered in double_scalars
  err = 200.*total/(hyp_tokens+ref_tokens)


In [16]:
x = ["work", "eight", "recognize speech" ]   
y = ["sword", "weights", "wreck a nice beach"]
#
print("String Matching using Weighted Edit Distance")
wS=1.1
wI=1.0
wD=1.0
print("Processing sentence by sentence")
for xx,yy in zip(x,y):
    print(xx,yy,wS,wI,wD)
    _,results = eva.edit_distance(xx,yy,wI=wI,wD=wD,wS=wS)
    print(results)
    
print("\n======================= ")
print("Corpus Processing")
_,results = eva.edit_distance(x,y,ALIGN=True,wI=wI,wD=wD,wS=wS)
eva.pp_results(results)
assert(results['total']==13)

String Matching using Weighted Edit Distance
Processing sentence by sentence
work sword 1.1 1.0 1.0
{'total': 2, 'sub': 1, 'ins': 0, 'del': 1, 'edit_dist': 2.1, 'err': 40.0, 'hyp_tokens': 4, 'ref_tokens': 5, 'utterances': 1}
eight weights 1.1 1.0 1.0
{'total': 2, 'sub': 0, 'ins': 0, 'del': 2, 'edit_dist': 2.0, 'err': 28.571428571428573, 'hyp_tokens': 5, 'ref_tokens': 7, 'utterances': 1}
recognize speech wreck a nice beach 1.1 1.0 1.0
{'total': 9, 'sub': 5, 'ins': 1, 'del': 3, 'edit_dist': 9.499999999999998, 'err': 50.0, 'hyp_tokens': 16, 'ref_tokens': 18, 'utterances': 1}

Corpus Processing
Error Rate: 43.33% 
Error Details: #S=6 #I=1 #D=6
Edit Distance: 13.60 
Tokens (HYP): 25    (REF): 30 
Utterances: 3


In [17]:
y = ["ik hou er van naar luister- en kamer-muziek te luisteren","blackboards and filterbanks are "]
x = ["ik hou ervan naar luister en kamer muziek te blijven luisteren","black boards and filter banks"]
#
print("Weighted Edit Distance with Compounding for Corpus")
#x = [eva.tokenizer(utt,TOKEN="WORD") for utt in tst]
#y = [eva.tokenizer(utt,TOKEN="WORD") for utt in ref]
_,results = \
    eva.edit_distance(x,y,ALIGN=True,TOKEN="WORD",CMPND=['','-'],VERBOSE=False,wI=2,wD=1,wS=1.3)
#
print("\nSUMMARY OF RESULTS\n==================")
eva.pp_results(results)
print("\nALIGNMENTS\n===============")
for al in results['align']: print(al) # eva.print_align(al)
assert(round(results['err'],2)==21.43)

Weighted Edit Distance with Compounding for Corpus

SUMMARY OF RESULTS
Error Rate: 21.43% 
Error Details: #S=1 #I=1 #D=1
Accepted Compounds: #C=4
Edit Distance: 5.10 
Tokens (HYP): 16    (REF): 14 
Utterances: 2

ALIGNMENTS
[('ik', 'ik'), ('hou', 'hou'), ('ervan', 'er+van'), ('naar', 'naar'), ('luister', 'luister-'), ('en', 'en'), ('kamer+muziek', 'kamer-muziek'), ('te', 'te'), ('blijven', '_'), ('luisteren', 'luisteren')]
[('black+boards', 'blackboards'), ('and', 'and'), ('filter+banks', 'filterbanks'), ('_', 'are')]


In [18]:
al = eva.align(x,y,TOKEN="WORD",CMPND=['','-'],EPS='<>') # \u03bs = small eps char; \u2205 = empty symbol set char')
al

[[('ik', 'ik'),
  ('hou', 'hou'),
  ('ervan', 'er+van'),
  ('naar', 'naar'),
  ('luister', 'luister-'),
  ('en', 'en'),
  ('kamer+muziek', 'kamer-muziek'),
  ('te', 'te'),
  ('blijven', '<>'),
  ('luisteren', 'luisteren')],
 [('black+boards', 'blackboards'),
  ('and', 'and'),
  ('filter+banks', 'filterbanks'),
  ('<>', 'are')]]